# Clean up

Lo que se pretende es aplicar los conceptos vistos de "Análisis Exploratorio" para limpiar los distintos set de datos proporcionados.

Archivos en: https://www.kaggle.com/benhamner/sf-bay-area-bike-share

Los archivos disponibles son 4:
    * station.csv (8 KB)
    * weather.csv (428 KB)
    * trip.csv    (77 M)
    * status.csv  (1,9 GB)

In [2]:
import pandas as pd

## Station.csv

In [3]:
station = pd.read_csv('station.csv', sep=',', parse_dates=['installation_date'],
                      infer_datetime_format=True,low_memory=False)

### Types

In [4]:
station.dtypes

id                            int64
name                         object
lat                         float64
long                        float64
dock_count                    int64
city                         object
installation_date    datetime64[ns]
dtype: object

### Null values

In [5]:
station.isnull().any()

id                   False
name                 False
lat                  False
long                 False
dock_count           False
city                 False
installation_date    False
dtype: bool

### Size

In [6]:
station.shape

(70, 7)

### Sample

In [7]:
station.head()

,id,name,lat,long,dock_count,city,installation_date
0,2,San Jose Diridon Caltrain Station,37.329732,-121.901782,27,San Jose,2013-08-06
1,3,San Jose Civic Center,37.330698,-121.888979,15,San Jose,2013-08-05
2,4,Santa Clara at Almaden,37.333988,-121.894902,11,San Jose,2013-08-06
3,5,Adobe on Almaden,37.331415,-121.893200,19,San Jose,2013-08-05
4,6,San Pedro Square,37.336721,-121.894074,15,San Jose,2013-08-07


## Weather.csv

In [8]:
weather = pd.read_csv('weather.csv', sep=',', parse_dates=['date'],
                      infer_datetime_format=True,low_memory=False)

### Types

In [9]:
weather.dtypes

date                              datetime64[ns]
max_temperature_f                        float64
mean_temperature_f                       float64
min_temperature_f                        float64
max_dew_point_f                          float64
mean_dew_point_f                         float64
min_dew_point_f                          float64
max_humidity                             float64
mean_humidity                            float64
min_humidity                             float64
max_sea_level_pressure_inches            float64
mean_sea_level_pressure_inches           float64
min_sea_level_pressure_inches            float64
max_visibility_miles                     float64
mean_visibility_miles                    float64
min_visibility_miles                     float64
max_wind_Speed_mph                       float64
mean_wind_speed_mph                      float64
max_gust_speed_mph                       float64
precipitation_inches                      object
cloud_cover         

In [10]:
weather['precipitation_inches'].unique()

array(['0', '0.23', 'T', '0.01', '0.28', '0.63', '0.29', '0.06', '0.85',
       '0.09', '0.64', '0.42', '0.35', '0.43', '0.22', '0.74', '0.03',
       '0.12', '0.16', '0.49', '0.17', '0.08', '0.04', '0.53', '0.07',
       '0.02', '0.83', '1.06', '1.71', '0.37', '0.27', '0.45', '0.78',
       '0.88', '0.66', '0.47', '0.1', '0.61', '0.14', '0.05', '0.68',
       '0.97', '0.26', '0.15', '0.87', '0.57', '0.69', '0.32', '0.21',
       '0.24', '0.52', '0.36', '0.33', '0.25', '0.11', '0.2', '1.18',
       '1.43', '3.12', '0.48', '0.19', '1.09', '0.65', '0.13', '0.91',
       '0.99', '0.18', '0.4', '1.07', nan, '0.41', '0.34', '1.25', '1.85',
       '3.36', '0.71', '1.3', '0.72', '0.6', '0.51', '1.2', '1.28', '3.23',
       '0.55', '1.26', '0.39'], dtype=object)

Llamativamente, "T", es un dato válido, por "trace", significa que se detectó lluvia, pero no la suficiente para poder ser medida.

[Fuente 1](http://help.wunderground.com/knowledgebase/articles/656875-what-does-t-stand-for-on-the-rain-precipitation)

Aquí, [Fuente 2](http://www.experts123.com/q/what-does-the-t-mean-in-the-precipitation-column-of-the-data-listing.html) indica que la precipitación debe ser menor a 0,01 pulgadas

En principio, opino que deberíamos mantener ese valor 'T', ya veremos si nos resulta de utilidad

In [11]:
weather[weather['precipitation_inches'] == 'T']['events'].unique()

array(['Fog', 'Rain', nan, 'Fog-Rain', 'Rain-Thunderstorm'], dtype=object)

Efectivamente, los eventos muestran que fueron días de lluvia, o al menos de humedad debido a la presencia de niebla.

In [12]:
weather['min_dew_point_f'] = pd.to_numeric(weather['min_dew_point_f'], errors='coerce')

In [13]:
weather['events'].unique()

array([nan, 'Fog', 'Rain', 'Fog-Rain', 'rain', 'Rain-Thunderstorm'], dtype=object)

In [14]:
# Uniformizando

weather['events'] = weather['events'].apply(lambda x: 'Rain' if x == 'rain' else x)

### Null values

In [15]:
weather.isnull().any()

date                              False
max_temperature_f                  True
mean_temperature_f                 True
min_temperature_f                  True
max_dew_point_f                    True
mean_dew_point_f                   True
min_dew_point_f                    True
max_humidity                       True
mean_humidity                      True
min_humidity                       True
max_sea_level_pressure_inches      True
mean_sea_level_pressure_inches     True
min_sea_level_pressure_inches      True
max_visibility_miles               True
mean_visibility_miles              True
min_visibility_miles               True
max_wind_Speed_mph                 True
mean_wind_speed_mph                True
max_gust_speed_mph                 True
precipitation_inches               True
cloud_cover                        True
events                             True
wind_dir_degrees                   True
zip_code                          False
dtype: bool

### Size

In [16]:
weather.shape

(3665, 24)

### Sample

In [40]:
weather.head()

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
0,2013-08-29,74.0,68.0,61.0,61.0,58.0,56.0,93.0,75.0,57.0,...,10.0,10.0,23.0,11.0,28.0,0,4.0,NaN,286.0,94107
1,2013-08-30,78.0,69.0,60.0,61.0,58.0,56.0,90.0,70.0,50.0,...,10.0,7.0,29.0,13.0,35.0,0,2.0,NaN,291.0,94107
2,2013-08-31,71.0,64.0,57.0,57.0,56.0,54.0,93.0,75.0,57.0,...,10.0,10.0,26.0,15.0,31.0,0,4.0,NaN,284.0,94107
3,2013-09-01,74.0,66.0,58.0,60.0,56.0,53.0,87.0,68.0,49.0,...,10.0,10.0,25.0,13.0,29.0,0,4.0,NaN,284.0,94107
4,2013-09-02,75.0,69.0,62.0,61.0,60.0,58.0,93.0,77.0,61.0,...,10.0,6.0,23.0,12.0,30.0,0,6.0,NaN,277.0,94107


## Trip.csv

In [18]:
trips = pd.read_csv('trip.csv', sep=',', parse_dates=['start_date','end_date'],
                      infer_datetime_format=True,low_memory=False)

### Types

In [19]:
trips.dtypes

id                             int64
duration                       int64
start_date            datetime64[ns]
start_station_name            object
start_station_id               int64
end_date              datetime64[ns]
end_station_name              object
end_station_id                 int64
bike_id                        int64
subscription_type             object
zip_code                      object
dtype: object

### Null values

In [20]:
trips.isnull().any()

id                    False
duration              False
start_date            False
start_station_name    False
start_station_id      False
end_date              False
end_station_name      False
end_station_id        False
bike_id               False
subscription_type     False
zip_code               True
dtype: bool

### Outliers

In [44]:
print(set(trips.zip_code.unique()))

set([nan, '35235', '110000', '11542', '11545', '5980', '8100003', '32506', '55343', '74012', '82225', '6790', '98311', '98312', '97150', '98641', '99999', '19971', '94847', '88022', '97333', '97331', '97330', '70866050', '60090', '17257', '10709', '10708', '10701', '26121', '63028', '10706', '10705', '84123', '84120', '84121', '84124', '84128', '84129', '45111', '45116', '85615', '44241', '44240', '81230170', '50219', '22877', '49306', '38501', '94155', '94153', '94150', '3513', '75050', '3510', '75056', '94158', '3514', '70663', '90640', '70665', '14624', '14627', '14620', '14623', '99', '68283', '38642', '91', '90', '92', '94', '18707', '95923', '78660', '18708', '78666', '78665', '94541', '94542', '94545', '94544', '94547', '94546', '94549', '17821', '87109', '87108', '20259', '20251', '87105', '87107', '87106', '83616', '23059', '23508', '23505', '1179', '27617', '27616', '27615', '27614', '27613', '27612', '941009', '38440', '560016', '29103', '7183', '88030300', '72704', '72701',

Los zip codes parecen cualquier ser cualquier cosa, deberían estar entre [estos valores](http://www.city-data.com/zipmaps/San-Francisco-California.html). De todos modos, a priori no me parece un dato que nos vaya a resultar útil. Tampoco sé si habla de zip de la estación inicial o final, en Kaggle no encontré información al respecto.

### Size

In [21]:
trips.shape

(669959, 11)

### Sample

In [23]:
trips.head()

,id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code
0,4576,63,2013-08-29 14:13:00,South Van Ness at Market,66,2013-08-29 14:14:00,South Van Ness at Market,66,520,Subscriber,94127
1,4607,70,2013-08-29 14:42:00,San Jose City Hall,10,2013-08-29 14:43:00,San Jose City Hall,10,661,Subscriber,95138
2,4130,71,2013-08-29 10:16:00,Mountain View City Hall,27,2013-08-29 10:17:00,Mountain View City Hall,27,48,Subscriber,97214
3,4251,77,2013-08-29 11:29:00,San Jose City Hall,10,2013-08-29 11:30:00,San Jose City Hall,10,26,Subscriber,95060
4,4299,83,2013-08-29 12:02:00,South Van Ness at Market,66,2013-08-29 12:04:00,Market at 10th,67,319,Subscriber,94103
